In [15]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [2]:
stack = 'MD589'
first, last = section_range_lookup[stack]

In [ ]:
# label_polygon = defaultdict(lambda: {})

# for sec, dm in dms.iteritems():
    
#     res = dm.load_proposal_review_result(None, 'latest', 'consolidated')
#     if res is None:
#         continue
#     usr, ts, suffix, result = res
#     for props in result:
#         label_polygon[props['label']][sec] = props['vertices']
    
# label_polygon.default_factory = None

# pickle.dump(label_polygon, open(os.environ['REPO_DIR']+'/database/'+stack+'_label_polygons.pkl', 'w'))

In [3]:
label_polygon = pickle.load(open(os.environ['REPO_DIR']+'/database/'+stack+'_label_polygons.pkl', 'r'))

labels_to_collect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

assert set(labels_to_collect).issubset(set(label_polygon.keys()))

In [4]:
section_contains_labels = defaultdict(set)
for l, secs in label_polygon.iteritems():
    for s in secs:
        section_contains_labels[s].add(l)
section_contains_labels.default_factory = None

In [5]:
patch_size = 224
half_size = patch_size/2
stride = 56

dm = DataManager(stack=stack, section=first, segm_params_id='tSLIC200', load_mask=False)

w = dm.image_width
h = dm.image_height 

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    
sample_locations = np.c_[xs.flat, ys.flat]

In [16]:
from shapely.geometry import Polygon, Point
from shapely.ops import cascaded_union

In [17]:
margin = int(.3*half_size)

In [20]:
sample_locations_ul = sample_locations - (margin, margin)
sample_locations_ur = sample_locations - (-margin, margin)
sample_locations_ll = sample_locations - (margin, -margin)
sample_locations_lr = sample_locations - (-margin, -margin)

In [24]:
# import time
# t = time.time()

all_indices = defaultdict(dict)

for sec, labels in section_contains_labels.iteritems():

# sec = 155
# labels = section_contains_labels[sec]

    print sec

    labs = set(labels) & set(labels_to_collect)

    if len(labs) == 0:
        continue

    dm = DataManager(stack=stack, section=sec, segm_params_id='tSLIC200', load_mask=False)

    mask = dm.load_thumbnail_mask()
    indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
    indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

    all_polygons = {}
    indices_inside = {}
    indices_buffer = {}

    # dm._load_image(['rgb'])
#     dm._load_image(['rgb-jpg'])

    for label in labs:

        path = Path(label_polygon[label][sec])
        indices_inside[label] =  np.where(path.contains_points(sample_locations_ll) &\
                                          path.contains_points(sample_locations_lr) &\
                                          path.contains_points(sample_locations_ul) &\
                                          path.contains_points(sample_locations_ur))[0]

    indices_allInside = np.concatenate(indices_inside.values())
        
    for label in labs:
 
        surround = Polygon(label_polygon[label][sec]).buffer(500, resolution=2)

        path = Path(list(surround.exterior.coords))
        indices_buf =  np.where(path.contains_points(sample_locations_ll) &\
                                path.contains_points(sample_locations_lr) &\
                                path.contains_points(sample_locations_ul) &\
                                path.contains_points(sample_locations_ur))[0]

        indices_buffer[label] = np.setdiff1d(indices_buf, np.r_[indices_bg, indices_allInside])

#         if len(indices_inside[label]) > 0:

#             output_dir = '/home/yuncong/CSHL_data_patches/%s/%04d/%s'%(stack, sec, label)
#             if not os.path.exists(output_dir):
#                 os.makedirs(output_dir)

#     #         for i in indices_inside[label]:
#             for i in np.random.choice(indices_inside[label], min(10, len(indices_inside[label])), replace=False):
#                 x, y = sample_locations[i]
#                 patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
#                 cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

#     #         for i in indices_buffer[label]:
#         if len(indices_buffer[label]) > 0:

#             output_dir = '/home/yuncong/CSHL_data_patches/%s/%04d/%s_surround'%(stack, sec, label)
#             if not os.path.exists(output_dir):
#                 os.makedirs(output_dir)

#             for i in np.random.choice(indices_buffer[label], min(10, len(indices_buffer[label])), replace=False):
#                 x, y = sample_locations[i]
#                 patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
#                 cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

#     output_dir = '/home/yuncong/CSHL_data_patches/%s/%04d/bg'%(stack, sec)
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for i in np.random.choice(indices_bg, 10, replace=False):
#         x, y = sample_locations[i]
#         patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
#         cv2.imwrite(output_dir + '/%08d.jpg'%i, img_as_ubyte(patch[..., ::-1]))

    all_indices[sec] = {}
    for l, inds in indices_inside.iteritems():
        all_indices[sec][l] = inds
    for l, inds in indices_buffer.iteritems():
        all_indices[sec][l+'_surround'] = inds

all_indices.default_factory = None
        
# print time.time() - t

111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
223
225
227
229
231
233
235
237
239
241
243
245
247
249
251
253
255
257
259
261
263
265
267
269
271
273
275
277
279
281
283
285
287
289
291
293
295
297
299
300
302
304
306
308
310
312
314
316
318
320
322
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351


In [23]:
all_indices

{'7N': array([20564, 20769, 20770, 20771, 20975, 20976, 20977, 21181, 21182,
        21183, 21387, 21388, 21389, 21593, 21594, 21595, 21596, 21799,
        21800, 21801, 21802, 22005, 22006, 22007, 22008, 22212, 22213,
        22214, 22419, 22420, 22626]),
 '7N_surround': array([18709, 18710, 18711, 18913, 18914, 18915, 18916, 18917, 18918,
        18919, 18920, 18921, 19117, 19118, 19119, 19120, 19121, 19122,
        19123, 19124, 19125, 19126, 19127, 19128, 19322, 19323, 19324,
        19325, 19326, 19327, 19328, 19329, 19330, 19331, 19332, 19333,
        19334, 19527, 19528, 19529, 19530, 19531, 19532, 19533, 19534,
        19535, 19536, 19537, 19538, 19539, 19540, 19541, 19732, 19733,
        19734, 19735, 19736, 19737, 19738, 19739, 19740, 19741, 19742,
        19743, 19744, 19745, 19746, 19747, 19748, 19938, 19939, 19940,
        19941, 19942, 19943, 19944, 19945, 19946, 19947, 19948, 19949,
        19950, 19951, 19952, 19953, 19954, 19955, 20143, 20144, 20145,
        20146, 201

In [22]:
all_indices = dict([])
for l, inds in indices_inside.iteritems():
    all_indices[l] = inds
for l, inds in indices_buffer.iteritems():
    all_indices[l+'_surround'] = inds
    
import cPickle as pickle
pickle.dump(all_indices, open('%(stack)s_%(sec)04d_indices.pkl'%{'stack':stack, 'sec':sec}, 'w'))

In [ ]:
# Whole slide

sec = 195
output_dir = '/home/yuncong/CSHL_data_patches/%s/%04d/%s'%(stack, sec, label)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in indices_fg:
    x, y = sample_locations[i]
#             patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
    patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
    cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

In [7]:
# ROI

box_x = 37
box_y = 122
box_w = 393
box_h = 219
xmin = box_x*32
xmax = (box_x+box_w)*32
ymin = box_y*32
ymax = (box_y+box_h)*32

sec = 195

dm = DataManager(stack=stack, section=sec, segm_params_id='tSLIC200', load_mask=False)

mask = dm.load_thumbnail_mask()
indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

# dm._load_image(['rgb'])
dm._load_image(['rgb-jpg'])

output_dir = '/home/yuncong/CSHL_data_patches/%s_byROI/%04d/roi1'%(stack, sec)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                    sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

indices_roi = np.setdiff1d(indices_roi, indices_bg)

print len(indices_roi), 'patches'

# for i in indices_roi:
# # for i in np.random.choice(indices_roi, min(10, len(indices_roi)), replace=False):
#     x, y = sample_locations[i]
# #             patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
#     patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
#     cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

22401 patches


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (157463552 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [8]:
for i in indices_roi:
# for i in np.random.choice(indices_roi, min(10, len(indices_roi)), replace=False):
    x, y = sample_locations[i]
#             patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
    patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
    cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

In [13]:
print 'tar -I pigz -C %(stack)s_byROI/%(sec)04d/roi1 -cf %(stack)s_%(sec)04d_roi1.tar .'% {'stack': stack, 'sec': sec}

tar -I pigz -C MD589_byROI/0195/roi1 -cf MD589_0195_roi1.tar .


In [ ]:
# copy patches to byLandmark folder

bg_dir = '/home/yuncong/CSHL_data_patches/%s_byLandmark/BackG/'%stack
if not os.path.exists(bg_dir):
    os.makedirs(bg_dir)
    
for l in labels_to_collect:
    output_dir = '/home/yuncong/CSHL_data_patches/%s_byLandmark/%s/'%(stack, l)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

for f in os.listdir('/home/yuncong/CSHL_data_patches/%s'%(stack)):
    os.chdir('/home/yuncong/CSHL_data_patches/%s'%(stack))
    for s in os.listdir('.'):
        for l in os.listdir(s):
            if len(l.split('_')) == 2:
                os.system('cp /home/yuncong/CSHL_data_patches/%s/%04d/%s/*'%(stack, int(s), l) + ' ' + bg_dir)
            else:
                output_dir = '/home/yuncong/CSHL_data_patches/%s_byLandmark/%s/'%(stack, l)
                os.system('cp /home/yuncong/CSHL_data_patches/%s/%04d/%s/*'%(stack, int(s), l) + ' ' + output_dir)

In [14]:
# Count number of patches each landmark

patches_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'
labels = os.listdir(patches_dir + stack + '_byLandmark')
for l in labels:
    print l, len(os.listdir(patches_dir + stack + '_byLandmark/' + l))

BackG 2668
7N 430
Gr 350
SuVe 80
Pn 505
VLL 260
LVe 110
12N 200
7n 500
5N 230


In [ ]:
# from joblib import delayed, Parallel

# import time
# t = time.time()

# _ = Parallel(n_jobs=16)(delayed(os.system)('cp %(f1)s %(f2)s'%{'f1': output_dir + '/%08d.jpg'%i,
#                                             'f2': output_dir2 + '/%08d.jpg'%i}) for i in reduced_sample_indices)

# print time.time() - t